# Setup

* Obtain some documents that are [supported by llamaindex](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/)
* Put those documents into a single directory in the same directory as this notebook.
* Change `UNIVERSE` to be your directory name
* Change `MODEL` to be a model you installed locally with [ollama](https://ollama.com)
* Run the pip cell, then the next cell.

To index four novels it took my M3 Macbook Pro about an hour.

In [ ]:
%pip install ipywidgets llama-index llama-index-llms-ollama llama-index-embeddings-ollama EbookLib html2text

In [ ]:
import os

from llama_index.core import load_index_from_storage, Settings, Document, SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.embeddings.ollama import OllamaEmbedding

UNIVERSE="universe1"
MODEL="llama3.2:3b-instruct-q8_0"

ollama_embedding = OllamaEmbedding(
    model_name=MODEL,
    base_url="http://localhost:11434",
)

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

Settings.embed_model=ollama_embedding

VECTOR_STORE_DISK=f"{UNIVERSE}.{MODEL}.index"
if not os.path.exists(VECTOR_STORE_DISK):
    reader = SimpleDirectoryReader(input_dir=UNIVERSE, recursive=True)
    documents = reader.load_data()
    nodes = node_parser.get_nodes_from_documents(documents)
    index = VectorStoreIndex(nodes, show_progress=True)
    index.storage_context.persist(VECTOR_STORE_DISK)

index = load_index_from_storage(StorageContext.from_defaults(persist_dir=VECTOR_STORE_DISK))
endpoint = index.as_retriever(similarity_top_k=10)


Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

In [107]:
response = endpoint.retrieve("The rapturous beauty of nature")
for i in range(len(response)):
    print(f"{i+1}. " +response[i].text.replace("\n", " "))

1. You may think them wonderful, but I have seen a greater wonder in this land, more beautiful than any grove or glade that ever grew: my heart is still full of it.  
2. Not only was there wonderful sunshine and delicious rain, in due times and perfect measure, but there seemed something more: an air of richness and growth, and a gleam of a beauty beyond that of mortal summers that flicker and pass upon this Middle- earth. 
3. In the Party Field a beautiful young sapling leaped up: it had silver bark and long leaves and burst into golden flowers in April. 
4. It is a rich and pleasant land, and its grass has no rival; but in these evil days folk do not dwell by the River or ride often to its shores. 
5. And they observed the winds and the air, and the matters of which Arda was made, of iron and stone and silver and gold and many substances: but of all these water they most greatly praised. 
6. Leaf and branch, water and stone: they have the hue and beauty of all these things under the 